<a href="https://colab.research.google.com/github/RrePitso/customer_retention_predictive_analytics/blob/main/Customer_Retention_Predictive_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
demographics = pd.read_excel('/content/drive/MyDrive/Customer_Churn/Customer_Churn_Data_Large.xlsx', sheet_name='Customer_Demographics')
churn = pd.read_excel('/content/drive/MyDrive/Customer_Churn/Customer_Churn_Data_Large.xlsx', sheet_name='Churn_Status')
transactions = pd.read_excel('/content/drive/MyDrive/Customer_Churn/Customer_Churn_Data_Large.xlsx', sheet_name='Transaction_History')
online = pd.read_excel('/content/drive/MyDrive/Customer_Churn/Customer_Churn_Data_Large.xlsx', sheet_name='Online_Activity')
service = pd.read_excel('/content/drive/MyDrive/Customer_Churn/Customer_Churn_Data_Large.xlsx', sheet_name='Customer_Service')

In [15]:
service.head()

,CustomerID,InteractionID,InteractionDate,InteractionType,ResolutionStatus
0,1,6363,2022-03-31,Inquiry,Resolved
1,2,3329,2022-03-17,Inquiry,Resolved
2,3,9976,2022-08-24,Inquiry,Resolved
3,4,7354,2022-11-18,Inquiry,Resolved
4,4,5393,2022-07-03,Inquiry,Unresolved


In [13]:
# Aggregate Customer_Service: count interactions, unresolved issues
service_agg = service.groupby('CustomerID').agg({
    'InteractionID': 'count',
    'ResolutionStatus': lambda x: (x == 'Unresolved').sum()
}).reset_index().rename(columns={
    'InteractionID': 'TotalInteractions',
    'ResolutionStatus': 'UnresolvedIssues'
})

# Clean AmountSpent (handle comma decimal)
transactions['AmountSpent'] = transactions['AmountSpent'].astype(str).str.replace(',', '.').astype(float)


# Aggregate Transaction_History: average spend per customer
transaction_agg = transactions.groupby('CustomerID').agg({
    'AmountSpent': 'mean'
}).reset_index().rename(columns={'AmountSpent': 'AvgSpent'})



In [14]:
service_agg.head()

,CustomerID,TotalInteractions,UnresolvedIssues
0,1,1,0
1,2,1,0
2,3,1,0
3,4,2,1
4,6,1,0
